# Delta calculator

A simple delta calculator

In [1]:
import pandas as pd
import numpy as np
import py_vollib.black_scholes.greeks.numerical as greeks
import py_vollib.black_scholes.implied_volatility as iv
import py_vollib.black_scholes as bs
from typing import Tuple

In [2]:
# parameters
spot:float = 17000
strike:float = 17000
risk_free_rate:float = 0.1
iv_range: Tuple[float, float] = (0.15,0.25)
step:float = 50 
de:int = 1 # days to expiry



In [3]:
kwargs = {
    'flag': 'c',
    'S': spot,
    'K': strike,
    't': de/360,
    'r': risk_free_rate
    
}

In [4]:
ivs = np.linspace(iv_range[0], iv_range[-1], 10)
strikes = set() 
for i in range(5):
    strikes.add((int(spot/step)*step) + i*step)
    strikes.add((int(spot/step)*step) - i*step)
spot_vals = np.linspace(spot*0.98,spot*1.02,10)
    


In [5]:
calls = []
puts = []
for sigma in ivs:
    for strk in strikes:
        kwargs.update({'sigma': sigma, 'K': strk, 'S': spot, 'flag': 'c'})
        price = bs.black_scholes(**kwargs)
        grk = greeks.delta(**kwargs)
        calls.append((sigma, strk, price, grk))
        kwargs.update({'flag': 'p'})
        price = bs.black_scholes(**kwargs)
        grk = greeks.delta(**kwargs)
        puts.append((sigma, strk, price, grk))
        

## Call option Delta (by strike)

**Call option delta for a fixed spot price with various strikes and implied volatility**

In [6]:
df1 = pd.DataFrame(calls, columns=['iv', 'strike', 'price', 'greek']).round(4)
df1_pivot = df1.pivot(index='iv', columns='strike', values='greek').round(4)
df1_pivot

strike,16800,16850,16900,16950,17000,17050,17100,17150,17200
iv,,,,,,,,,
0.1500,0.9377,0.8770,0.7839,0.6597,0.5156,0.3698,0.2411,0.1418,0.0749
0.1611,0.9237,0.8601,0.7678,0.6495,0.5147,0.3787,0.2566,0.1592,0.0900
0.1722,0.9097,0.8441,0.7534,0.6404,0.5140,0.3865,0.2706,0.1755,0.1050
0.1833,0.8959,0.8291,0.7403,0.6325,0.5134,0.3934,0.2832,0.1906,0.1196
0.1944,0.8824,0.8152,0.7284,0.6254,0.5129,0.3996,0.2946,0.2047,0.1337
0.2056,0.8694,0.8021,0.7176,0.6190,0.5124,0.4052,0.3049,0.2177,0.1472
0.2167,0.8569,0.7899,0.7077,0.6133,0.5120,0.4102,0.3143,0.2299,0.1601
0.2278,0.8449,0.7786,0.6987,0.6081,0.5116,0.4147,0.3230,0.2411,0.1723
0.2389,0.8336,0.7680,0.6904,0.6035,0.5113,0.4188,0.3309,0.2516,0.1839


## Put option Delta (by strike)
**Put option delta for a fixed spot price with various strikes and implied volatility**

In [7]:
df2 = pd.DataFrame(puts, columns=['iv', 'strike', 'price', 'greek']).round(4)
df2_pivot = df2.pivot(index='iv', columns='strike', values='greek').round(4)
df2_pivot

strike,16800,16850,16900,16950,17000,17050,17100,17150,17200
iv,,,,,,,,,
0.1500,-0.0623,-0.1230,-0.2161,-0.3403,-0.4844,-0.6302,-0.7589,-0.8582,-0.9251
0.1611,-0.0763,-0.1399,-0.2322,-0.3505,-0.4853,-0.6213,-0.7434,-0.8408,-0.9100
0.1722,-0.0903,-0.1559,-0.2466,-0.3596,-0.4860,-0.6135,-0.7294,-0.8245,-0.8950
0.1833,-0.1041,-0.1709,-0.2597,-0.3675,-0.4866,-0.6066,-0.7168,-0.8094,-0.8804
0.1944,-0.1176,-0.1848,-0.2716,-0.3746,-0.4871,-0.6004,-0.7054,-0.7953,-0.8663
0.2056,-0.1306,-0.1979,-0.2824,-0.3810,-0.4876,-0.5948,-0.6951,-0.7823,-0.8528
0.2167,-0.1431,-0.2101,-0.2923,-0.3867,-0.4880,-0.5898,-0.6857,-0.7701,-0.8399
0.2278,-0.1551,-0.2214,-0.3013,-0.3919,-0.4884,-0.5853,-0.6770,-0.7589,-0.8277
0.2389,-0.1664,-0.2320,-0.3096,-0.3965,-0.4887,-0.5812,-0.6691,-0.7484,-0.8161


# Combined Delta - fixed spot

In [8]:
pvt = (df1_pivot + df2_pivot).round(4)
pvt.style.background_gradient(cmap='RdYlGn', axis=None)

strike,16800,16850,16900,16950,17000,17050,17100,17150,17200
iv,,,,,,,,,
0.15,0.875400,0.754000,0.567800,0.319400,0.031200,-0.260400,-0.517800,-0.716400,-0.850200
0.1611,0.847400,0.720200,0.535600,0.299000,0.029400,-0.242600,-0.486800,-0.681600,-0.820000
0.1722,0.819400,0.688200,0.506800,0.280800,0.028000,-0.227000,-0.458800,-0.649000,-0.790000
0.1833,0.791800,0.658200,0.480600,0.265000,0.026800,-0.213200,-0.433600,-0.618800,-0.760800
0.1944,0.764800,0.630400,0.456800,0.250800,0.025800,-0.200800,-0.410800,-0.590600,-0.732600
0.2056,0.738800,0.604200,0.435200,0.238000,0.024800,-0.189600,-0.390200,-0.564600,-0.705600
0.2167,0.713800,0.579800,0.415400,0.226600,0.024000,-0.179600,-0.371400,-0.540200,-0.679800
0.2278,0.689800,0.557200,0.397400,0.216200,0.023200,-0.170600,-0.354000,-0.517800,-0.655400
0.2389,0.667200,0.536000,0.380800,0.207000,0.022600,-0.162400,-0.338200,-0.496800,-0.632200


In [9]:
calls = []
puts = []
for sigma in ivs:
    for spt in spot_vals:
        kwargs.update({'sigma': sigma, 'S': spt, 'K':strike, 'flag': 'c'})
        price = bs.black_scholes(**kwargs)
        grk = greeks.delta(**kwargs)
        calls.append((sigma, spt, price, grk))
        kwargs.update({'flag': 'p'})
        price = bs.black_scholes(**kwargs)
        grk = greeks.delta(**kwargs)
        puts.append((sigma, spt, price, grk))

## Call option delta (by spot)

**Call option delta for a fixed strike price with various spot prices and implied volatility**


In [10]:
df3 = pd.DataFrame(calls, columns=['iv', 'spot', 'price', 'greek']).round(4)
df3['spot'] = df3.spot.astype(int)
df3_pivot = df3.pivot(index='iv', columns='spot', values='greek').round(4)
df3_pivot

spot,16660,16735,16811,16886,16962,17037,17113,17188,17264,17340
iv,,,,,,,,,,
0.1500,0.0059,0.0259,0.0847,0.2098,0.4043,0.6255,0.8105,0.9246,0.9768,0.9945
0.1611,0.0096,0.0352,0.1005,0.2264,0.4110,0.6173,0.7937,0.9096,0.9682,0.9911
0.1722,0.0142,0.0453,0.1159,0.2414,0.4169,0.6101,0.7785,0.8948,0.9587,0.9867
0.1833,0.0198,0.0560,0.1308,0.2550,0.4220,0.6038,0.7646,0.8804,0.9486,0.9814
0.1944,0.0262,0.0671,0.1450,0.2675,0.4267,0.5983,0.7519,0.8666,0.9380,0.9753
0.2056,0.0333,0.0784,0.1586,0.2788,0.4308,0.5933,0.7403,0.8534,0.9273,0.9685
0.2167,0.0410,0.0897,0.1715,0.2892,0.4346,0.5888,0.7297,0.8408,0.9165,0.9611
0.2278,0.0491,0.1008,0.1836,0.2987,0.4379,0.5847,0.7199,0.8288,0.9057,0.9534
0.2389,0.0575,0.1118,0.1951,0.3075,0.4410,0.5811,0.7109,0.8175,0.8951,0.9453


## Put option delta (by spot)

**Put option delta for a fixed strike price with various spot prices and implied volatility**


In [11]:
df4 = pd.DataFrame(puts, columns=['iv', 'spot', 'price', 'greek']).round(4)
df4['spot'] = df4.spot.astype(int)
df4_pivot = df4.pivot(index='iv', columns='spot', values='greek').round(4)
df4_pivot

spot,16660,16735,16811,16886,16962,17037,17113,17188,17264,17340
iv,,,,,,,,,,
0.1500,-0.9941,-0.9741,-0.9153,-0.7902,-0.5957,-0.3745,-0.1895,-0.0754,-0.0232,-0.0055
0.1611,-0.9904,-0.9648,-0.8995,-0.7736,-0.5890,-0.3827,-0.2063,-0.0904,-0.0318,-0.0089
0.1722,-0.9858,-0.9547,-0.8841,-0.7586,-0.5831,-0.3899,-0.2215,-0.1052,-0.0413,-0.0133
0.1833,-0.9802,-0.9440,-0.8692,-0.7450,-0.5780,-0.3962,-0.2354,-0.1196,-0.0514,-0.0186
0.1944,-0.9738,-0.9329,-0.8550,-0.7325,-0.5733,-0.4017,-0.2481,-0.1334,-0.0620,-0.0247
0.2056,-0.9667,-0.9216,-0.8414,-0.7212,-0.5692,-0.4067,-0.2597,-0.1466,-0.0727,-0.0315
0.2167,-0.9590,-0.9103,-0.8285,-0.7108,-0.5654,-0.4112,-0.2703,-0.1592,-0.0835,-0.0389
0.2278,-0.9509,-0.8992,-0.8164,-0.7013,-0.5621,-0.4153,-0.2801,-0.1712,-0.0943,-0.0466
0.2389,-0.9425,-0.8882,-0.8049,-0.6925,-0.5590,-0.4189,-0.2891,-0.1825,-0.1049,-0.0547


# Combined Delta - fixed strike

Combined delta for a specific strike but with different volatility and spot prices

In [12]:
pvt = (df3_pivot + df4_pivot).round(4)
pvt.style.background_gradient(cmap='RdYlGn', axis=None)

spot,16660,16735,16811,16886,16962,17037,17113,17188,17264,17340
iv,,,,,,,,,,
0.15,-0.988200,-0.948200,-0.830600,-0.580400,-0.191400,0.251000,0.621000,0.849200,0.953600,0.989000
0.1611,-0.980800,-0.929600,-0.799000,-0.547200,-0.178000,0.234600,0.587400,0.819200,0.936400,0.982200
0.1722,-0.971600,-0.909400,-0.768200,-0.517200,-0.166200,0.220200,0.557000,0.789600,0.917400,0.973400
0.1833,-0.960400,-0.888000,-0.738400,-0.490000,-0.156000,0.207600,0.529200,0.760800,0.897200,0.962800
0.1944,-0.947600,-0.865800,-0.710000,-0.465000,-0.146600,0.196600,0.503800,0.733200,0.876000,0.950600
0.2056,-0.933400,-0.843200,-0.682800,-0.442400,-0.138400,0.186600,0.480600,0.706800,0.854600,0.937000
0.2167,-0.918000,-0.820600,-0.657000,-0.421600,-0.130800,0.177600,0.459400,0.681600,0.833000,0.922200
0.2278,-0.901800,-0.798400,-0.632800,-0.402600,-0.124200,0.169400,0.439800,0.657600,0.811400,0.906800
0.2389,-0.885000,-0.776400,-0.609800,-0.385000,-0.118000,0.162200,0.421800,0.635000,0.790200,0.890600
